# <center> Análisis exploratorio de los datos </center>

In [1]:
%%configure -f
{
  "conf": {
    "spark.sql.catalogImplementation": "hive",
    "hive.metastore.client.factory.class": "com.amazonaws.glue.catalog.metastore.AWSGlueDataCatalogHiveClientFactory",
    "spark.hadoop.hive.metastore.client.factory.class": "com.amazonaws.glue.catalog.metastore.AWSGlueDataCatalogHiveClientFactory",
    "spark.sql.warehouse.dir": "s3://s3-pi-perfilamiento-20252/warehouse/",
    "spark.yarn.appMasterEnv.AWS_REGION": "us-east-1",
    "spark.executorEnv.AWS_REGION": "us-east-1",
    "spark.hadoop.fs.s3a.aws.credentials.provider": "com.amazonaws.auth.InstanceProfileCredentialsProvider"
  }
}

## Librerías

In [35]:
import seaborn as sns
import matplotlib.pyplot as plt
from pyspark.sql import SparkSession
from pyspark.sql.functions import countDistinct, avg, col, sum as spark_sum

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## Lectura de datos

In [5]:
database_name = "proyecto1db"
table_name = "trusted"

spark_df = spark.table(f"{database_name}.{table_name}")
spark_df.show(5)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+---------+-----------------+----------+---------------------+-------------------+----------------+-------+---------------+--------+------------------------+--------+-------------------+----+------------+------------+--------------------+-------------------+-----+--------------+
|codcliente|codpoliza|formapagovigencia|valortotal|valor_total_periodico|fechainiciovigencia|fechafinvigencia| estado|opracionesgrupo|vigencia|estadorenovacionesinicio|  ciudad|    tipovinculacion|edad|tipo_empresa|     tomador|         nomproducto|nomgrupoempresarial| ramo|orden_vigencia|
+----------+---------+-----------------+----------+---------------------+-------------------+----------------+-------+---------------+--------+------------------------+--------+-------------------+----+------------+------------+--------------------+-------------------+-----+--------------+
|      NULL|     NULL|formaPagoVigencia|valorTotal| Valor_Total_Perio...|fechaInicioVigencia|fechaFinVigencia| estado|Opracione

## Información general

In [6]:
# Número total de registros
print(f"Número total de registros: {spark_df.count()}")

# Esquema del DataFrame
spark_df.printSchema()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

N?mero total de registros: 81759
root
 |-- codcliente: long (nullable = true)
 |-- codpoliza: long (nullable = true)
 |-- formapagovigencia: string (nullable = true)
 |-- valortotal: string (nullable = true)
 |-- valor_total_periodico: string (nullable = true)
 |-- fechainiciovigencia: string (nullable = true)
 |-- fechafinvigencia: string (nullable = true)
 |-- estado: string (nullable = true)
 |-- opracionesgrupo: string (nullable = true)
 |-- vigencia: string (nullable = true)
 |-- estadorenovacionesinicio: string (nullable = true)
 |-- ciudad: string (nullable = true)
 |-- tipovinculacion: string (nullable = true)
 |-- edad: long (nullable = true)
 |-- tipo_empresa: string (nullable = true)
 |-- tomador: string (nullable = true)
 |-- nomproducto: string (nullable = true)
 |-- nomgrupoempresarial: string (nullable = true)
 |-- ramo: string (nullable = true)
 |-- orden_vigencia: long (nullable = true)

## Estadísticas descriptivas

In [23]:
# Estadísticas descriptivas para columnas numéricas
numeric_types = ['int', 'double', 'float', 'bigint', 'long']
num_cols = [c for c, t in spark_df.dtypes if t in numeric_types]
spark_df.select(num_cols).describe().show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+------------------+-----------------+------------------+--------------+
|summary|        codcliente|        codpoliza|              edad|orden_vigencia|
+-------+------------------+-----------------+------------------+--------------+
|  count|             81758|            81758|             81758|         81457|
|   mean|128796.03280412925|124621.3013405416| 27.69540595415739|           1.0|
| stddev| 76173.51297032544|71950.65781835375|24.629428069159708|           0.0|
|    min|                37|                9|                 0|             1|
|    max|            267485|           227235|                99|             1|
+-------+------------------+-----------------+------------------+--------------+

## Exploración de datos

In [24]:
# Contar valores nulos por columna
spark_df.select([spark_sum(col(c).isNull().cast("integer")).alias(c) for c in spark_df.columns]).show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+---------+-----------------+----------+---------------------+-------------------+----------------+------+---------------+--------+------------------------+------+---------------+----+------------+-------+-----------+-------------------+----+--------------+
|codcliente|codpoliza|formapagovigencia|valortotal|valor_total_periodico|fechainiciovigencia|fechafinvigencia|estado|opracionesgrupo|vigencia|estadorenovacionesinicio|ciudad|tipovinculacion|edad|tipo_empresa|tomador|nomproducto|nomgrupoempresarial|ramo|orden_vigencia|
+----------+---------+-----------------+----------+---------------------+-------------------+----------------+------+---------------+--------+------------------------+------+---------------+----+------------+-------+-----------+-------------------+----+--------------+
|         1|        1|                0|         0|                    0|                  0|               0|     0|              0|       0|                       0|     0|              0|   

In [25]:
# Contar registros por 'Ramo'
spark_df.groupBy('Ramo').count().show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------+-----+
|         Ramo|count|
+-------------+-----+
|Patrimoniales|10116|
|        Otros|  313|
|         Vida|19287|
|      Allianz|  301|
| Cumplimiento|11450|
|        Autos|18531|
|        Salud|21760|
|         Ramo|    1|
+-------------+-----+

In [26]:
# Contar registros por 'nomGrupoEmpresarial'
spark_df.groupBy('nomGrupoEmpresarial').count().show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+-----+
| nomGrupoEmpresarial|count|
+--------------------+-----+
|                 Hdi| 2575|
|             Bolivar| 9586|
|             Allianz| 4920|
|                 Sbs|  477|
|                Sura|61417|
|          La Equidad|   41|
|           Solidaria| 1588|
|             Mundial|   44|
|                Bbva|   13|
|              Mapfre|  126|
|           Deudores"|  297|
|           Previsora|   74|
|           Colpatria|  164|
|            S Estado|  190|
|    Modalidad Ideal"|    1|
| Modalidad Deudores"|    3|
|           48 Hooras|    4|
|              Zurich|    8|
|                    |  217|
| nomGrupoEmpresarial|    1|
+--------------------+-----+
only showing top 20 rows

## Matriz de correlación

In [38]:
# Calcular la matriz de correlación para columnas numéricas
numeric_cols = [col for col, dtype in spark_df.dtypes if dtype in numeric_types]
corr_matrix = spark_df.select(numeric_cols).toPandas().corr()

# Visualizar la matriz de correlación
plt.figure(figsize=(10, 8))
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', fmt=".2f")
plt.title('Correlation Matrix of Numerical Columns')
plt.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## Gráficos de dispersión

In [40]:
sample_spark_df = spark_df.select(numeric_cols).limit(1000).toPandas()

sns.pairplot(sample_spark_df)
plt.suptitle('Pair Plot of Numerical Columns (Sampled Data)', y=1.02)
plt.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## Promedio de registros por cliente, registros por póliza y pólizas por cliente

In [41]:
# Promedio de registros por cliente
avg_records_per_client_spark = spark_df.groupBy('codCliente').count().select(avg('count')).first()[0]
print(f"Promedio de registros por cliente: {avg_records_per_client_spark:.2f}")

# Promedio de registros por póliza
avg_records_per_poliza_spark = spark_df.groupBy('codPoliza').count().select(avg('count')).first()[0]
print(f"Promedio de registros por póliza: {avg_records_per_poliza_spark:.2f}")

# Promedio de pólizas por cliente
avg_policies_per_client_spark = spark_df.groupBy('codCliente').agg(countDistinct('codPoliza').alias('policy_count')).select(avg('policy_count')).first()[0]
print(f"Promedio de pólizas por cliente: {avg_policies_per_client_spark:.2f}")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Promedio de registros por cliente: 1.58
Promedio de registros por p?liza: 1.00
Promedio de p?lizas por cliente: 1.58